In [1]:
import numpy as np
import pandas as pd

In [2]:
num_individuals = 100000
choices = ['a', 'b', 'c', 'd', 'e']

rng = np.random.default_rng(20230329)
individuals = pd.DataFrame(
    10 * rng.standard_normal(size=(num_individuals, len(choices))),
    columns=choices
)
ranked = []
for i in range(len(choices)):
    individuals[f'choice_{i+1}'] = individuals.iloc[:, :len(choices)].apply(lambda row: row.nlargest(i+1).values[-1],axis=1)
    ranked.append(f'choice_{i+1}')
individuals.head()

,a,b,c,d,e,choice_1,choice_2,choice_3,choice_4,choice_5
0,25.862944,-0.944372,4.584851,13.773822,-19.125875,25.862944,13.773822,4.584851,-0.944372,-19.125875
1,-9.115143,5.967015,13.390357,-25.291344,2.975015,13.390357,5.967015,2.975015,-9.115143,-25.291344
2,0.509622,-11.576557,0.610888,-7.391570,0.591932,0.610888,0.591932,0.509622,-7.391570,-11.576557
3,5.984562,-2.305313,-6.715193,-12.998302,-8.590250,5.984562,-2.305313,-6.715193,-8.590250,-12.998302
4,6.495212,-14.286467,6.501225,-2.854895,7.028091,7.028091,6.501225,6.495212,-2.854895,-14.286467


In [3]:
df = pd.DataFrame(individuals[ranked].mean(), columns=['mean_utility'])
df['mean_diff_utility'] = df['mean_utility'] - df['mean_utility'].shift(-1)
df

,mean_utility,mean_diff_utility
choice_1,11.642013,6.691560
choice_2,4.950454,4.940664
choice_3,0.009790,4.928750
choice_4,-4.918960,6.660227
choice_5,-11.579187,NaN


In [4]:
print(df)

          mean_utility  mean_diff_utility
choice_1     11.642013           6.691560
choice_2      4.950454           4.940664
choice_3      0.009790           4.928750
choice_4     -4.918960           6.660227
choice_5    -11.579187                NaN


In [5]:
individuals_truncated = individuals[choices[:-1]].copy()
ranked = []
for i in range(len(choices[:-1])):
    individuals_truncated[f'choice_{i+1}'] = individuals_truncated.iloc[:, :len(choices[:-1])].apply(lambda row: row.nlargest(i+1).values[-1],axis=1)
    ranked.append(f'choice_{i+1}')
individuals_truncated.head()

,a,b,c,d,choice_1,choice_2,choice_3,choice_4
0,25.862944,-0.944372,4.584851,13.773822,25.862944,13.773822,4.584851,-0.944372
1,-9.115143,5.967015,13.390357,-25.291344,13.390357,5.967015,-9.115143,-25.291344
2,0.509622,-11.576557,0.610888,-7.391570,0.610888,0.509622,-7.391570,-11.576557
3,5.984562,-2.305313,-6.715193,-12.998302,5.984562,-2.305313,-6.715193,-12.998302
4,6.495212,-14.286467,6.501225,-2.854895,6.501225,6.495212,-2.854895,-14.286467


In [6]:
df_truncated = pd.DataFrame(individuals_truncated[ranked].mean(), columns=['mean_utility'])
df_truncated['mean_diff_utility'] = df_truncated['mean_utility'] - df_truncated['mean_utility'].shift(-1)
df_truncated

,mean_utility,mean_diff_utility
choice_1,10.312395,7.325077
choice_2,2.987318,5.915780
choice_3,-2.928463,7.296400
choice_4,-10.224863,NaN


In [7]:
print(df_truncated)

          mean_utility  mean_diff_utility
choice_1     10.312395           7.325077
choice_2      2.987318           5.915780
choice_3     -2.928463           7.296400
choice_4    -10.224863                NaN


In [8]:
votes_4 = pd.DataFrame(
    [
        [.48, .52, 0, 0],
        [.52, .48, 0, 0],
        [0, 0, .48, .52],
        [0, 0, .52, .48],        
    ]
)
votes_4

,0,1,2,3
0,0.48,0.52,0.00,0.00
1,0.52,0.48,0.00,0.00
2,0.00,0.00,0.48,0.52
3,0.00,0.00,0.52,0.48


In [9]:
weights = df_truncated.reset_index()[['mean_utility']]
weights

,mean_utility
0,10.312395
1,2.987318
2,-2.928463
3,-10.224863


In [10]:
results_4 = votes_4 @ weights
results_4.index = choices[:4]
results_4

,mean_utility
a,6.503355
b,6.796358
c,-6.722591
d,-6.430735


In [11]:
votes_4@weights

,mean_utility
0,6.503355
1,6.796358
2,-6.722591
3,-6.430735


In [12]:
votes_5 = pd.DataFrame(
    [
        [.48, 0, 1.0, 0, 0],
        [0, 1.000, 0, 0, 0],
        [0, 0, .48, 0, .52],
        [0, 0, 0, 1.000, 0],
        [.52, 0, 0, 0, .48]
    ]
)
votes_5

,0,1,2,3,4
0,0.48,0.0,1.00,0.0,0.00
1,0.00,1.0,0.00,0.0,0.00
2,0.00,0.0,0.48,0.0,0.52
3,0.00,0.0,0.00,1.0,0.00
4,0.52,0.0,0.00,0.0,0.48


In [13]:
weights = df.reset_index()[['mean_utility']]
weights

,mean_utility
0,11.642013
1,4.950454
2,0.009790
3,-4.918960
4,-11.579187


In [14]:
results_5 = votes_5 @ weights
results_5.index = choices
results_5

,mean_utility
a,5.597956
b,4.950454
c,-6.016478
d,-4.918960
e,0.495837
